In [1]:
#!/usr/bin/env Rscript
#setwd('analysis')
##########################
library(scales)
library(plyr)
library(Seurat)
library(dplyr)
library(patchwork)
library(ggplot2)
library(cowplot)
library("viridis")  
library(RColorBrewer)
library("viridis")
library(Matrix)
library(EnhancedVolcano)
library(writexl)

Attaching SeuratObject


다음의 패키지를 부착합니다: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



다음의 패키지를 부착합니다: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots


필요한 패키지를 로딩중입니다: viridisLite


다음의 패키지를 부착합니다: ‘viridis’


The following object is masked from ‘package:scales’:

    viridis_pal


필요한 패키지를 로딩중입니다: ggrepel



In [13]:
# Reading Datafile and Metadata

data=readRDS('../Data_BBB/brain.BBB.human.vascular.final.Jan2022.rds')
meta=readRDS('../Data_BBB/brain.BBB.human.vascular.final.Jan2022.metadata.rds')

brain=CreateSeuratObject(counts = data, project = "BBB", meta.data=meta,min.cells = 3, min.features = 200)

Warning message in CreateSeuratObject.default(counts = data, project = "BBB", meta.data = meta, :
“Some cells in meta.data not present in provided counts matrix”


In [14]:
# Make CD19 expression Label 
CD19exp <- FetchData(object = brain, vars = c('CD19'))
CD19label <- lapply(CD19exp, function(x) {
  ifelse(x > 0, "CD19_pos", "CD19_neg")
})
  
brain$CD19exp <- CD19label



In [15]:
# Normalization and featuring

brain.list <- SplitObject(brain, split.by = "CD19exp")

brain.list <- lapply(X = brain.list, FUN = function(x) {
    x <- NormalizeData(brain, normalization.method = "LogNormalize", scale.factor = 10000)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
})

features <- SelectIntegrationFeatures(object.list = brain.list)

In [16]:
brain.anchors <- FindIntegrationAnchors(object.list = brain.list, anchor.features = features)
brain.combined <- IntegrateData(anchorset = brain.anchors)


Warning message in CheckDuplicateCellNames(object.list = object.list):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”
Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 14499 anchors

Filtering anchors

	Retained 12373 anchors

Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data



In [17]:
DefaultAssay(brain.combined) <- "integrated"

# Run the standard workflow for visualization and clustering
brain.combined <- ScaleData(brain.combined, verbose = FALSE)
brain.combined <- RunPCA(brain.combined, npcs = 30, verbose = FALSE)
brain.combined <- RunUMAP(brain.combined, reduction = "pca", dims = 1:30)
brain.combined <- FindNeighbors(brain.combined, reduction = "pca", dims = 1:30)
brain.combined <- FindClusters(brain.combined, resolution = 0.5)


17:57:33 UMAP embedding parameters a = 0.9922 b = 1.112

17:57:33 Read 9936 rows and found 30 numeric columns

17:57:33 Using Annoy for neighbor search, n_neighbors = 30

17:57:33 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

17:57:35 Writing NN index file to temp file /var/folders/jx/rq_jr02135l_jt9wtyd2x47c0000gn/T//RtmpKubUou/file127081a1449

17:57:35 Searching Annoy index using 1 thread, search_k = 3000

17:57:38 Annoy recall = 100%

17:57:38 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

17:57:39 Initializing from normalized Laplacian + noise (using irlba)

17:57:40 Commencing optimization for 500 epochs, with 420098 positive edges

17:58:05 Optimization finished

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 9936
Number of edges: 345462

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9347
Number of communities: 20
Elapsed time: 1 seconds


In [18]:
DefaultAssay(brain.combined) <- "RNA"
nk.markers <- FindConservedMarkers(brain.combined, grouping.var = "CD19exp", verbose = FALSE)
head(nk.markers)

ERROR: Error in paste(ident.1, level.use, sep = "_"): 기본값이 없는 인수 "ident.1"가 누락되어 있습니다


In [49]:
theme_set(theme_cowplot())

# Fib 1552
subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Fib1" |  cellsubtype == "Fib2"  |  cellsubtype == "Fib3")
title <- "Fibroblasts"
#SMC
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "aSMC" |  cellsubtype == "vSMC")
#title <- "SMC"
# Endo 4426
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "capEndo" |  cellsubtype == "aEndo"  |  cellsubtype == "vEndo")
#title <- "Endo"
# Per   3128
# subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Per1" |  cellsubtype == "Per2"  |  cellsubtype == "Per3")
# title <- "Perycites"
#subpopul.cells
length(which(subpopul.cells$CD19exp=='CD19_pos'))

Idents(subpopul.cells) <- "CD19exp"
avg.subpopul.cells <- as.data.frame(log1p(AverageExpression(subpopul.cells, verbose = FALSE)$RNA))
avg.subpopul.cells$gene <- rownames(avg.subpopul.cells)

markers <- FindMarkers(subpopul.cells, ident.1 = "CD19_pos", ident.2 = "CD19_neg")
head(markers)

#ggsave(paste("VolcanoPlot_fullrange",title,"png"))

df <- head(markers)
df$Genes <- rownames(df)

write_xlsx(df, paste0("../Figures/VolcanoHead_",title,".xlsx"))

options(repr.plot.width=12, repr.plot.height=10)
jpeg(file=paste0("../Figures/VolcanoPlot_",title,".jpeg"))


p2<-EnhancedVolcano(markers , rownames(markers),
                x ="avg_log2FC", 
                y ="p_val",
                title = title,
        pCutoff = 1e-4,
        FCcutoff = 1,
        pointSize = 3.0,
        labSize = 6.0)

p2 + ylim(0, 8)

dev.off()

[1] 4

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CD19,0.000000e+00,1.6098461,1.0,0.000,0.000000e+00
TNF,2.427944e-170,0.9498107,0.5,0.000,6.083942e-166
AC007349.1,2.427944e-170,1.0844546,0.5,0.000,6.083942e-166
ERICD,4.789471e-86,0.9466452,0.5,0.001,1.200146e-81
CCDC13-AS1,1.304000e-85,1.0743215,0.5,0.001,3.267564e-81
AL355581.1,1.304000e-85,1.0798460,0.5,0.001,3.267564e-81


ERROR: Error: (경고에서 변환) One or more p-values is 0. Converting to 10^-1 * current lowest non-zero p-value...


In [ ]:
theme_set(theme_cowplot())

# Fib 1552
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Fib1" |  cellsubtype == "Fib2"  |  cellsubtype == "Fib3")
#title <- "Fibroblasts"
#SMC
subpopul.cells <- subset(brain.combined, subset = cellsubtype == "aSMC" |  cellsubtype == "vSMC")
title <- "SMC"
# Endo 4426
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "capEndo" |  cellsubtype == "aEndo"  |  cellsubtype == "vEndo")
#title <- "Endo"
# Per   3128
# subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Per1" |  cellsubtype == "Per2"  |  cellsubtype == "Per3")
# title <- "Perycites"
#subpopul.cells
length(which(subpopul.cells$CD19exp=='CD19_pos'))

Idents(subpopul.cells) <- "CD19exp"
avg.subpopul.cells <- as.data.frame(log1p(AverageExpression(subpopul.cells, verbose = FALSE)$RNA))
avg.subpopul.cells$gene <- rownames(avg.subpopul.cells)

markers <- FindMarkers(subpopul.cells, ident.1 = "CD19_pos", ident.2 = "CD19_neg")
head(markers)

#ggsave(paste("VolcanoPlot_fullrange",title,"png"))

df <- head(markers)
df$Genes <- rownames(df)

write_xlsx(df, paste0("../Figures/VolcanoHead_",title,".xlsx"))

options(repr.plot.width=12, repr.plot.height=10)
jpeg(file=paste0("../Figures/VolcanoPlot_",title,".jpeg"))


p2<-EnhancedVolcano(markers , rownames(markers),
                x ="avg_log2FC", 
                y ="p_val",
                title = title,
        pCutoff = 1e-4,
        FCcutoff = 1,
        pointSize = 3.0,
        labSize = 6.0)

p2 + ylim(0, 8)

dev.off()

In [ ]:
theme_set(theme_cowplot())

# Fib 1552
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Fib1" |  cellsubtype == "Fib2"  |  cellsubtype == "Fib3")
#title <- "Fibroblasts"
#SMC
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "aSMC" |  cellsubtype == "vSMC")
#title <- "SMC"
# Endo 4426
subpopul.cells <- subset(brain.combined, subset = cellsubtype == "capEndo" |  cellsubtype == "aEndo"  |  cellsubtype == "vEndo")
title <- "Endo"
# Per   3128
# subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Per1" |  cellsubtype == "Per2"  |  cellsubtype == "Per3")
# title <- "Perycites"
#subpopul.cells
length(which(subpopul.cells$CD19exp=='CD19_pos'))

Idents(subpopul.cells) <- "CD19exp"
avg.subpopul.cells <- as.data.frame(log1p(AverageExpression(subpopul.cells, verbose = FALSE)$RNA))
avg.subpopul.cells$gene <- rownames(avg.subpopul.cells)

markers <- FindMarkers(subpopul.cells, ident.1 = "CD19_pos", ident.2 = "CD19_neg")
head(markers)

#ggsave(paste("VolcanoPlot_fullrange",title,"png"))

df <- head(markers)
df$Genes <- rownames(df)

write_xlsx(df, paste0("../Figures/VolcanoHead_",title,".xlsx"))

options(repr.plot.width=12, repr.plot.height=10)
jpeg(file=paste0("../Figures/VolcanoPlot_",title,".jpeg"))


p2<-EnhancedVolcano(markers , rownames(markers),
                x ="avg_log2FC", 
                y ="p_val",
                title = title,
        pCutoff = 1e-4,
        FCcutoff = 1,
        pointSize = 3.0,
        labSize = 6.0)

p2 + ylim(0, 8)

dev.off()

In [ ]:
theme_set(theme_cowplot())

# Fib 1552
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Fib1" |  cellsubtype == "Fib2"  |  cellsubtype == "Fib3")
#title <- "Fibroblasts"
#SMC
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "aSMC" |  cellsubtype == "vSMC")
#title <- "SMC"
# Endo 4426
#subpopul.cells <- subset(brain.combined, subset = cellsubtype == "capEndo" |  cellsubtype == "aEndo"  |  cellsubtype == "vEndo")
#title <- "Endo"
# Per   3128
subpopul.cells <- subset(brain.combined, subset = cellsubtype == "Per1" |  cellsubtype == "Per2"  |  cellsubtype == "Per3")
title <- "Perycites"
#subpopul.cells
length(which(subpopul.cells$CD19exp=='CD19_pos'))

Idents(subpopul.cells) <- "CD19exp"
avg.subpopul.cells <- as.data.frame(log1p(AverageExpression(subpopul.cells, verbose = FALSE)$RNA))
avg.subpopul.cells$gene <- rownames(avg.subpopul.cells)

markers <- FindMarkers(subpopul.cells, ident.1 = "CD19_pos", ident.2 = "CD19_neg")
head(markers)

#ggsave(paste("VolcanoPlot_fullrange",title,"png"))

df <- head(markers)
df$Genes <- rownames(df)

write_xlsx(df, paste0("../Figures/VolcanoHead_",title,".xlsx"))

options(repr.plot.width=12, repr.plot.height=10)
jpeg(file=paste0("../Figures/VolcanoPlot_",title,".jpeg"))


p2<-EnhancedVolcano(markers , rownames(markers),
                x ="avg_log2FC", 
                y ="p_val",
                title = title,
        pCutoff = 1e-4,
        FCcutoff = 1,
        pointSize = 3.0,
        labSize = 6.0)

p2 + ylim(0, 8)

dev.off()

In [48]:
genes.to.label = c('CD19',c(rownames(monocyte.de.markers)[which(monocyte.de.markers$avg_log2FC>3)]))
p1 <- ggplot(avg.subpopul.cells, aes(CD19_neg, CD19_pos)) + geom_point() + ggtitle("Fib")
p1 <- LabelPoints(plot = p1, points = genes.to.label, repel = TRUE)
p1

ERROR: Error in rownames(monocyte.de.markers): 객체 'monocyte.de.markers'를 찾을 수 없습니다


In [47]:
table(brain@assays$RNA@data)

ERROR: Error in unique.default(x, nmax = nmax): unique()는 오로지 벡터들에만 적용됩니다


In [ ]:
#cd19.cells <- subset(brain, subset = CD19 > 0)

gene.correlations <- cor(brain@assays$RNA@data, FetchData(brain,vars='CD19'), method = "spearman")
sorted.genes <- sort(gene.correlations, decreasing = TRUE)
top.genes <- head(sorted.genes, 200)
library(org.Hs.eg.db)
top.genes.ids <- as.character(top.genes)
top.genes.entrez <- mapIds(org.Hs.eg.db, keys = top.genes.ids, keytype = "SYMBOL", column = "ENTREZID")
library(clusterProfiler)
enriched.go <- enrichGO(gene = top.genes.entrez, OrgDb = "org.Hs.eg.db", keyType = "ENTREZID")
dotplot(enriched.go, showCategory = 20)



In [ ]:
theme_set(theme_cowplot())

t.cells <- subset(brain.combined, subset = cellsubtype == "capEndo" |  cellsubtype == "aEndo"  |  cellsubtype == "vEndo")
Idents(t.cells) <- "CD19exp"
avg.t.cells <- as.data.frame(log1p(AverageExpression(t.cells, verbose = FALSE)$RNA))
avg.t.cells$gene <- rownames(avg.t.cells)

cd14.mono <- subset(brain.combined, subset = cellsubtype == "Per1" |  cellsubtype == "Per2"  |  cellsubtype == "Per3")
Idents(cd14.mono) <- "CD19exp"
avg.cd14.mono <- as.data.frame(log1p(AverageExpression(cd14.mono, verbose = FALSE)$RNA))
avg.cd14.mono$gene <- rownames(avg.cd14.mono)

genes.to.label = c('CD19')
p1 <- ggplot(avg.t.cells, aes(CD19_n, CD19_p)) + geom_point() + ggtitle("Endo")
p1 <- LabelPoints(plot = p1, points = genes.to.label, repel = TRUE)
p2 <- ggplot(avg.cd14.mono, aes(CD19_n, CD19_p)) + geom_point() + ggtitle("Per")
p2 <- LabelPoints(plot = p2, points = genes.to.label, repel = TRUE)
p1 + p2